# ACTIVIDAD APACHE SPARK - MÓDULO 7 SESIÓN 3

## 1. Configuración del Entorno

**Pasos de configuración:**
1. Instalar dependencias necesarias
2. Importar librerías necesarias
3. Crear SparkSession con configuraciones específicas
4. Verificar que la sesión esté activa

In [ ]:
# Instalar dependencias necesarias
!pip install pyspark requests

In [ ]:
# Importar librerías necesarias para el procesamiento
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import requests
import json

In [ ]:
# Crear sesión de Spark con configuraciones para procesamiento local
# PROPÓSITO: Establecer el contexto principal para todas las operaciones de Spark
spark = SparkSession.builder \
    .appName("ActividadModulo7Sesion3") \
    .master("local[*]") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

print("Configuración de Spark completada:")
print(f"Sesión Spark creada: {spark.sparkContext.appName}")
print(f"Modo de ejecución: {spark.sparkContext.master}")
print(f"Versión de Spark: {spark.version}")

Configuración de Spark completada:
Sesión Spark creada: ActividadModulo7Sesion3
Modo de ejecución: local[*]
Versión de Spark: 3.5.1


## 2. Obtención de Datos desde API Pública

**API seleccionada:** JSONPlaceholder
- URL: https://jsonplaceholder.typicode.com/posts
- Justificación: API gratuita que proporciona datos estructurados en JSON

In [ ]:
# Obtener datos estructurados desde una API REST pública

url_api = "https://jsonplaceholder.typicode.com/posts"
print(f"Obteniendo datos desde: {url_api}")

# Realizar petición HTTP GET para obtener los datos
respuesta = requests.get(url_api)

if respuesta.status_code == 200:
    datos_json = respuesta.json()
    print(f"{len(datos_json)} registros obtenidos exitosamente")

    # Mostrar estructura del primer registro
    print("\nEstructura de datos (primer registro):")
    for campo, valor in datos_json[0].items():
        print(f"  - {campo}: {type(valor).__name__}")

else:
    raise Exception(f"Error HTTP {respuesta.status_code}")

Obteniendo datos desde: https://jsonplaceholder.typicode.com/posts
100 registros obtenidos exitosamente

Estructura de datos (primer registro):
  - userId: int
  - id: int
  - title: str
  - body: str


In [ ]:
# Crear RDD desde los datos JSON
rdd_original = spark.sparkContext.parallelize(datos_json)

print(f"RDD creado con {rdd_original.count()} elementos")
print(f"Distribuido en {rdd_original.getNumPartitions()} particiones")

# Mostrar algunos registros para validar la carga
print("\nPrimeros 2 registros del RDD:")
for i, registro in enumerate(rdd_original.take(2)):
    print(f"Post {i+1}: userId={registro['userId']}, id={registro['id']}")

RDD creado con 100 elementos
Distribuido en 2 particiones

Primeros 2 registros del RDD:
Post 1: userId=1, id=1
Post 2: userId=1, id=2


## 3. Procesamiento de Datos - Aplicando Transformaciones

**Concepto clave:** Las transformaciones son operaciones que modifican los datos sin ejecutarse inmediatamente (lazy evaluation). Solo se ejecutan cuando se llama a una acción.

In [ ]:
# TRANSFORMACIÓNES

# TRANSFORMACIÓN 1: filter() - Filtrar registros específicos
# Filtrar posts de usuarios específicos para análisis enfocado

print("TRANSFORMACIÓN 1: filter()")
rdd_filtrado = rdd_original.filter(lambda post: 2 <= post['userId'] <= 7)

print(f"Posts antes del filtro: {rdd_original.count()}")
print(f"Posts después del filtro (userId 2-7): {rdd_filtrado.count()}")

# Mostrar ejemplo de datos filtrados
print("\nEjemplo de datos filtrados:")
for i, post in enumerate(rdd_filtrado.take(2)):
    print(f"  Post {i+1}: userId={post['userId']}, id={post['id']}")

TRANSFORMACIÓN 1: filter()
Posts antes del filtro: 100
Posts después del filtro (userId 2-7): 60

Ejemplo de datos filtrados:
  Post 1: userId=2, id=11
  Post 2: userId=2, id=12


In [ ]:
# TRANSFORMACIÓN 2: map() - Transformar estructura de datos
# Extraer métricas de contenido para análisis cuantitativo

print("TRANSFORMACIÓN 2: map()")

def extraer_metricas(post):
    """Extrae métricas relevantes de cada post para análisis"""
    titulo_palabras = len(post['title'].split())
    cuerpo_palabras = len(post['body'].split())

    return {
        'userId': post['userId'],
        'postId': post['id'],
        'titulo_palabras': titulo_palabras,
        'cuerpo_palabras': cuerpo_palabras,
        'total_palabras': titulo_palabras + cuerpo_palabras
    }

rdd_metricas = rdd_filtrado.map(extraer_metricas)

print("Ejemplo de métricas calculadas:")
for i, metrica in enumerate(rdd_metricas.take(2)):
    print(f"  Post {i+1}: {metrica['titulo_palabras']} palabras título, "
          f"{metrica['cuerpo_palabras']} palabras cuerpo, "
          f"total: {metrica['total_palabras']}")

TRANSFORMACIÓN 2: map()
Ejemplo de métricas calculadas:
  Post 1: 6 palabras título, 25 palabras cuerpo, total: 31
  Post 2: 6 palabras título, 26 palabras cuerpo, total: 32


In [ ]:
# TRANSFORMACIÓN 3: flatMap() - Descomponer datos en elementos individuales
# Extraer palabras individuales para análisis de vocabulario

print("TRANSFORMACIÓN 3: flatMap()")

def extraer_palabras(post):
    """Extrae todas las palabras del título convertidas a minúsculas"""
    return post['title'].lower().split()

rdd_palabras = rdd_filtrado.flatMap(extraer_palabras)

print(f"Total de palabras extraídas: {rdd_palabras.count()}")
print("Muestra de palabras extraídas:")
palabras_muestra = rdd_palabras.take(10)
print(f"  {palabras_muestra}")

TRANSFORMACIÓN 3: flatMap()
Total de palabras extraídas: 371
Muestra de palabras extraídas:
  ['et', 'ea', 'vero', 'quia', 'laudantium', 'autem', 'in', 'quibusdam', 'tempore', 'odit']


## 4. Aplicando Acciones - Materializar Resultados

In [ ]:
# ACCIÓN 1: collect() - Recopilar todos los resultados en memoria
# Traer todos los datos procesados al driver para análisis

print("ACCIÓN 1: collect()")
todas_las_metricas = rdd_metricas.collect()
print(f"{len(todas_las_metricas)} métricas recopiladas en memoria")

# Calcular estadísticas simples con los datos recopilados
total_palabras = sum(m['total_palabras'] for m in todas_las_metricas)
promedio_palabras = total_palabras / len(todas_las_metricas)

print(f"\nEstadísticas calculadas:")
print(f"  - Total palabras procesadas: {total_palabras}")
print(f"  - Promedio de palabras por post: {promedio_palabras:.1f}")

ACCIÓN 1: collect()
60 métricas recopiladas en memoria

Estadísticas calculadas:
  - Total palabras procesadas: 1809
  - Promedio de palabras por post: 30.1


In [ ]:
# ACCIÓN 2: reduce() - Aplicar función de reducción
# Encontrar el post con más contenido usando operación distribuida

print("ACCIÓN 2: reduce()")

def comparar_posts(post1, post2):
    """Compara dos posts y retorna el que tiene más palabras totales"""
    return post1 if post1['total_palabras'] > post2['total_palabras'] else post2

post_mas_largo = rdd_metricas.reduce(comparar_posts)

print(f"Post más largo encontrado:")
print(f"  - Post ID: {post_mas_largo['postId']}")
print(f"  - Usuario: {post_mas_largo['userId']}")
print(f"  - Total palabras: {post_mas_largo['total_palabras']}")
print(f"  - Palabras título: {post_mas_largo['titulo_palabras']}")
print(f"  - Palabras cuerpo: {post_mas_largo['cuerpo_palabras']}")

ACCIÓN 2: reduce()
Post más largo encontrado:
  - Post ID: 59
  - Usuario: 6
  - Total palabras: 41
  - Palabras título: 9
  - Palabras cuerpo: 32


## 5. Almacenamiento y Persistencia

**Propósito:** Optimizar el rendimiento almacenando RDDs que se usan múltiples veces

In [ ]:
# Almacenar RDD filtrado en cache para optimizar múltiples operaciones
# Evitar recálculo cuando el RDD se usa en varias transformaciones
rdd_filtrado.cache()
print(f"RDD filtrado almacenado en cache: {rdd_filtrado.is_cached}")

# Persistir métricas en disco para evitar recálculo
# Guardar resultados complejos en almacenamiento permanente
rdd_metricas.persist(StorageLevel.DISK_ONLY)
print(f"RDD métricas persistido: {rdd_metricas.getStorageLevel()}")

RDD filtrado almacenado en cache: True
RDD métricas persistido: Disk Serialized 1x Replicated


## 6. Análisis Final y Estadísticas

In [ ]:
# Agrupar métricas por usuario para análisis comparativo
# Generar estadísticas agregadas por usuario

def agrupar_por_usuario(metricas):
    usuarios = {}
    for m in metricas:
        uid = m['userId']
        if uid not in usuarios:
            usuarios[uid] = {'posts': 0, 'palabras_titulo': 0, 'palabras_cuerpo': 0}

        usuarios[uid]['posts'] += 1
        usuarios[uid]['palabras_titulo'] += m['titulo_palabras']
        usuarios[uid]['palabras_cuerpo'] += m['cuerpo_palabras']

    return usuarios

estadisticas_usuarios = agrupar_por_usuario(todas_las_metricas)

print("Estadísticas por usuario:")
for user_id, stats in estadisticas_usuarios.items():
    total_palabras = stats['palabras_titulo'] + stats['palabras_cuerpo']
    print(f"  Usuario {user_id}: {stats['posts']} posts, {total_palabras} palabras totales")

# Encontrar usuario más productivo
usuario_productivo = max(estadisticas_usuarios.items(),
                        key=lambda x: x[1]['palabras_titulo'] + x[1]['palabras_cuerpo'])

print(f"\nUsuario más productivo: {usuario_productivo[0]}")
print(f"Total palabras: {usuario_productivo[1]['palabras_titulo'] + usuario_productivo[1]['palabras_cuerpo']}")

Estadísticas por usuario:
  Usuario 2: 10 posts, 299 palabras totales
  Usuario 3: 10 posts, 293 palabras totales
  Usuario 4: 10 posts, 332 palabras totales
  Usuario 5: 10 posts, 305 palabras totales
  Usuario 6: 10 posts, 288 palabras totales
  Usuario 7: 10 posts, 292 palabras totales

Usuario más productivo: 4
Total palabras: 332


## 7. Limpieza de Recursos

In [ ]:
# Liberar cache y persistencia para optimizar memoria
# Limpiar recursos utilizados durante el procesamiento

print("Liberando cache y persistencia...")
rdd_filtrado.unpersist()
rdd_metricas.unpersist()

print(f"RDD filtrado en cache: {rdd_filtrado.is_cached}")
print("Recursos de almacenamiento liberados")

print("\n" + "=" * 60)
print("PROCESAMIENTO COMPLETADO EXITOSAMENTE")
print("Para detener Spark ejecutar: spark.stop()")
print("=" * 60)

Liberando cache y persistencia...
RDD filtrado en cache: False
Recursos de almacenamiento liberados

PROCESAMIENTO COMPLETADO EXITOSAMENTE
Para detener Spark ejecutar: spark.stop()


## Resumen Ejecutivo

In [ ]:
# Generar resumen final de la actividad
print(f"""
RESUMEN EJECUTIVO:
- API utilizada: JSONPlaceholder (posts públicos)
- Datos procesados: {len(datos_json)} posts de API
- Usuarios analizados: {len(estadisticas_usuarios)} usuarios (ID 2-7)
- Transformaciones aplicadas: 3 (filter, map, flatMap)
- Acciones ejecutadas: 2 (collect, reduce)
- Total palabras procesadas: {total_palabras}
- Usuario más productivo: {usuario_productivo[0]}
- Post más extenso: ID {post_mas_largo['postId']} ({post_mas_largo['total_palabras']} palabras)
""")


RESUMEN EJECUTIVO:
- API utilizada: JSONPlaceholder (posts públicos)
- Datos procesados: 100 posts de API
- Usuarios analizados: 6 usuarios (ID 2-7)
- Transformaciones aplicadas: 3 (filter, map, flatMap)  
- Acciones ejecutadas: 2 (collect, reduce)
- Total palabras procesadas: 292
- Usuario más productivo: 4
- Post más extenso: ID 59 (41 palabras)

